In [17]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [1]:
datafolder = ''

In [25]:
experiment = 'Eoi400'
model = 'IPSL-CM6A'

In [26]:
dsV = xr.open_dataset(datafolder+'Raw_data/'+model+'/'+experiment+'/vo_100yr.nc')
dsT = xr.open_dataset(datafolder+'Raw_data/'+model+'/'+experiment+'/thetao_100yr.nc')
dsS = xr.open_dataset(datafolder+'Raw_data/'+model+'/'+experiment+'/so_100yr.nc')

dsgrid = xr.open_dataset(datafolder+'/Raw_data/'+model+'/'+experiment+'/'+experiment+'.SST.mean_month.nc', decode_times=False)

In [27]:
dsV = dsV.rename({'nav_lon': 'ULONG', 'nav_lat': 'ULAT'})
dsT = dsT.rename({'nav_lon': 'TLONG', 'nav_lat': 'TLAT'})
dsS = dsS.rename({'nav_lon': 'TLONG', 'nav_lat': 'TLAT'})

In [28]:
regridder = xe.Regridder(dsS, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.so)

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.olevel.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

In [29]:
regridder = xe.Regridder(dsV, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.vo*100)

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.olevel.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

In [30]:
regridder = xe.Regridder(dsT, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.thetao)

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.olevel.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')